In [2]:
#Import thư viện cần thiết
from pdf2image import convert_from_path
import os
import re
import unicodedata
from collections import defaultdict
import difflib
import pandas as pd
from PIL import Image
import pytesseract
import cv2
import numpy as np
import matplotlib.pyplot as plt
from paddleocr import PaddleOCR


1. Chuyển file PDF sang các file ảnh

In [4]:
#Xử lí PDF 1 thành hình ảnh dạng png từ trang 203 đến trang 395 (theo id)
pdf_file = "../data/pdf/PDF1.pdf"
output_folder = "../image/PDF1"


images = convert_from_path(
    pdf_file,
    first_page=203,
    last_page=395
)

for page_num, img in enumerate(images, start=203):
    filename = f"PDF1-{page_num}.png"
    save_path = os.path.join(output_folder, filename)
    img.save(save_path, "PNG")


In [5]:
#Xử lí PDF 3 thành hình ảnh dạng png từ trang 12 đến trang 118 (theo id)
pdf_file = "../data/pdf/PDF3.pdf"
output_folder = "../image/PDF3"

os.makedirs(output_folder, exist_ok=True)

start_page = 12
end_page = 118

images = convert_from_path(pdf_file, first_page=start_page, last_page=end_page)

for page_num, img in zip(range(start_page, end_page + 1), images):
    page_str = f"{page_num:03d}"           
    filename = f"PDF3-{page_str}.png"
    save_path = os.path.join(output_folder, filename)

    img.save(save_path, "PNG")

2. Tiền xử lí ảnh trước khi OCR

In [8]:
# =====================================================
INPUT_ROOT = '../image'
INPUT_SUBDIR = 'PDF1'
INPUT_DIR = os.path.join(INPUT_ROOT, INPUT_SUBDIR)

OUTPUT_SUBDIR = 'PDF1_processed'
OUTPUT_DIR = os.path.join(INPUT_ROOT, OUTPUT_SUBDIR)

START_PAGE = 250
END_PAGE = 296

if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)
    print(f"✅ Đã tạo thư mục đầu ra: {OUTPUT_DIR}")

# =====================================================
# 2. LỌC FILE THEO SỐ TRANG (GIỮ NGUYÊN)
# =====================================================
target_files = []
pattern = re.compile(r'PDF1-(\d+)\.png$', re.IGNORECASE)

try:
    all_files = os.listdir(INPUT_DIR)
    for file_name in all_files:
        match = pattern.match(file_name)
        if match:
            page_number = int(match.group(1))
            if START_PAGE <= page_number <= END_PAGE:
                target_files.append(file_name)
except FileNotFoundError:
    print(f"❌ Không tìm thấy thư mục đầu vào: {INPUT_DIR}")

target_files.sort(key=lambda f: int(pattern.search(f).group(1)))

# =====================================================
# 3. HÀM XỬ LÝ TỐI ƯU OCR: CHỐNG MẤT NÉT & LÀM ĐẬM CHỮ
# =====================================================
def process_optimized_for_ocr(image_path):
    img = cv2.imread(image_path)
    if img is None:
        return None, None

    # 1. Chuyển sang Grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # 2. Tăng cường độ tương phản cục bộ (CLAHE)
    # Giúp các dòng chữ nhạt màu như "số 641" hiện rõ hơn khỏi nền
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    enhanced = clahe.apply(gray)

    # 3. Adaptive Thresholding (Phân ngưỡng thích nghi)
    # Thay vì dùng ngưỡng cố định cho cả ảnh, nó tính toán từng vùng nhỏ
    # giúp giữ lại nét chữ mảnh cực tốt.
    binary = cv2.adaptiveThreshold(
        enhanced, 255, 
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
        cv2.THRESH_BINARY, 21, 15
    )

    # 4. KHỬ NHIỄU NHẸ (Dùng Median Blur kích thước nhỏ nhất)
    # Để xóa các đốm đen li ti mà không làm hỏng cấu trúc chữ
    denoised = cv2.medianBlur(binary, 3)

    # 5. LÀM ĐẬM NÉT CHỮ (Erosion)
    # Với ảnh nền trắng chữ đen, phép Erode sẽ làm các vùng đen (chữ) dày lên.
    # Điều này cực kỳ quan trọng để Tesseract nhận diện được các chữ bị nhạt/đứt nét.
    kernel = np.ones((2, 2), np.uint8)
    final_img = cv2.erode(denoised, kernel, iterations=1)

    return final_img, img

# =====================================================
# 4. THỰC THI XỬ LÝ ĐỒNG LOẠT
# =====================================================

count = 0
for file_name in target_files:
    full_path = os.path.join(INPUT_DIR, file_name)
    processed_img, _ = process_optimized_for_ocr(full_path)

    if processed_img is not None:
        output_path = os.path.join(OUTPUT_DIR, f"processed_{file_name}")
        cv2.imwrite(output_path, processed_img)
        count += 1
        if count % 10 == 0:
            print(f" Đã xử lý {count}/{len(target_files)} ảnh")

print(f"\n  Đã lưu {count} ảnh vào: {OUTPUT_DIR}")

 Đã xử lý 10/47 ảnh
 Đã xử lý 20/47 ảnh
 Đã xử lý 30/47 ảnh
 Đã xử lý 40/47 ảnh

  Đã lưu 47 ảnh vào: ../image\PDF1_processed


3. Xử lí các ảnh thông qua sử dụng Paddle OCR và Teserract OCR. 

In [9]:
# OCR Tesseract
TESSERACT_LANGS = 'chi_sim+chi_tra+vie'
def tesseract_ocr(img_path: str, lang: str = TESSERACT_LANGS):
    try:
        text_raw = pytesseract.image_to_string(
            Image.open(img_path),
            lang=lang,
            config='--psm 3'
        )
        return [l.strip() for l in text_raw.split('\n') if l.strip()]
    except Exception as e:
        print(f"Lỗi OCR {img_path}: {e}")
        return []
    
# OCR Paddle
ocr_paddle = PaddleOCR(
    use_angle_cls=True,
    lang='ch',
    show_log=False
)

In [10]:
def strip_accents(s: str) -> str:
    return "".join(
        c for c in unicodedata.normalize("NFD", s)
        if unicodedata.category(c) != "Mn"
    )

def is_chinese(text: str) -> bool:
    if not text:
        return False
        
    pattern = (
        r'^['
        # Nhóm Hán tự
        r'\u4E00-\u9FFF'  # CJK Unified Ideographs (Phổ thông)
        r'\u3400-\u4DBF'  # CJK Extension A
        r'\uF900-\uFAFF'  # CJK Compatibility
        
        # Nhóm dấu câu CJK
        r'\u3000-\u303F'  # Dấu câu TQ (。 、 【 】...)
        r'\uFF00-\uFFEF'  # Dấu câu Full-width (！, ？, ０-９...)
        r'\uFE30-\uFE4F'  # CJK Compatibility Forms
        
        # Nhóm ASCII (không có kí tự Latin)
        r'\u0020-\u0040'  # Khoảng trắng, Dấu câu (!..@), SỐ (0-9)
        r'\u005B-\u0060'  # Dấu câu ( [ .. ` )
        r'\u007B-\u007E'  # Dấu câu ( { .. ~ )
        
        # Nhóm Latin-1 Suplement (Giữ lại để bắt dấu » « ©)
        # r'\u00A0-\u00FF'  
        r']+$'
    )
    
    return bool(re.fullmatch(pattern, text))

def is_page_number_line(text: str) -> bool:
    return re.fullmatch(r"^\s*\d{1,4}\s*$", text) is not None

def looks_like_vietnamese(text: str) -> bool:
    norm = strip_accents(text).lower()
    tokens = re.findall(r"[a-z0-9]+", norm)

    def has_similar(target: str, thr: float) -> bool:
        for t in tokens:
            if difflib.SequenceMatcher(None, t, target).ratio() >= thr:
                return True
        return False

    return (
        has_similar("buc", 0.5) and
        has_similar("thu", 0.5) and
        has_similar("viet", 0.5) and
        has_similar("cho", 0.5) and
        has_similar("chinh", 0.5) and
        has_similar("minh", 0.5)
    )

def extract_id(text: str):
    def norm_digits(tok: str):
        tok = tok.strip()
        tok = tok.replace('O', '0').replace('o', '0')
        tok = tok.replace('I', '1').replace('l', '1').replace('|', '1').replace('!', '1')

        digits = "".join(re.findall(r"\d+", tok))  # "50 4" -> "504"
        if not digits:
            return None
        if len(digits) < 3:
            return None
        return digits

    # Chinese: 第xxx 
    m = re.search(r"第\s*([0-9OoIl|!\s]{1,20})", text)
    if m:
        return norm_digits(m.group(1))

    # Vietnamese: so/s0/s6 ...
    norm = strip_accents(text).lower()
    m = re.search(r"(?:\bso\b|\bs0\b|\bs6\b|s[0o6])\s*([0-9OoIl|!\s]{1,20})", norm)
    if m:
        return norm_digits(m.group(1))

    return None

In [11]:
# Parse ảnh -> Data
DATA_MAP_TESSERACT = defaultdict(lambda: {"src": [], "tgt": [], "vi_started": False})
DATA_MAP_PADDLE = defaultdict(lambda: {"src": [], "tgt": [], "vi_started": False})
STATE_TESSERACT = {"current_id": None, "mode": None, "pending_src": []}
STATE_PADDLE = {"current_id": None, "mode": None, "pending_src": []}

def process_image_to_data(img_path: str, data_map, state, ocr_model):
    if ocr_model == 'Tesseract':
        lines = tesseract_ocr(img_path)
    else:
        result = ocr_paddle.ocr(img_path, cls=True)
        if not result or result[0] is None:
            return

        blocks = result[0]
        blocks = sorted(
            blocks,
            key=lambda b: (min(p[1] for p in b[0]), min(p[0] for p in b[0]))
        )
        lines = [b[1][0].strip() for b in blocks if b and b[1] and b[1][0]]

    current_id = state["current_id"]
    mode = state["mode"]
    pending_src = state["pending_src"]

    for text in lines:
        if not text:
            continue
        if is_page_number_line(text):
            continue
        
        # print(f"Model: {ocr_model}\n{text}")

        # Title tiếng Việt -> mode=vi (lấy ID) và chỉ xử lý như title nếu extract được ID
        if looks_like_vietnamese(text):
            maybe_id = extract_id(text)
            if maybe_id:
                # Title thật
                current_id = maybe_id
                _ = data_map[current_id]
                if pending_src:
                    data_map[current_id]["src"].extend(pending_src)
                    pending_src.clear()

                mode = "vi"
                if not data_map[current_id]["vi_started"]:
                    data_map[current_id]["tgt"] = []
                    data_map[current_id]["vi_started"] = True
                
                data_map[current_id]["tgt"].append(
                    f"Bức thư viết cho chính mình số {current_id}"
                )
                continue

        # Anchor ID (ưu tiên từ tiếng Trung: 第...)
        found_id = extract_id(text)
        if found_id:
            current_id = found_id
            _ = data_map[current_id]
            if pending_src:
                data_map[current_id]["src"].extend(pending_src)
                pending_src.clear()

            mode = "zh" if is_chinese(text) else "vi"
            if mode == "vi" and (not data_map[current_id]["vi_started"]):
                data_map[current_id]["tgt"] = []
                data_map[current_id]["vi_started"] = True
            
            data_map[current_id]["src"].append(
                f"写 给 自己 的 第 {current_id} 封 信"
            )
            continue

        # Chưa có ID -> giữ tiếng Trung để chờ, bỏ qua Pinyin
        if current_id is None:
            if is_chinese(text):
                pending_src.append(text)
            continue

        # Gom nội dung theo mode
        if is_chinese(text):
            if mode == "vi":
                # Gặp tiếng Trung sau tiếng Việt -> Reset để chờ ID mới
                pending_src.append(text)
                current_id = None
                mode = "zh"
            else:
                data_map[current_id]["src"].append(text)
        else:
            if mode == "vi":
                data_map[current_id]["tgt"].append(text)

    state["current_id"] = current_id
    state["mode"] = mode
    state["pending_src"] = pending_src

In [13]:
# Main process
IMG_DIR = "../image"
OUT_DIR = "../OCR"
os.makedirs(OUT_DIR, exist_ok=True)

for p in range(START_PAGE, END_PAGE + 1):
    img_path_tesseract = os.path.join(IMG_DIR, f"{INPUT_SUBDIR}_processed/processed_PDF1-{p}.png")
    img_path_paddle = os.path.join(IMG_DIR, f"{INPUT_SUBDIR}/PDF1-{p}.png")
    if os.path.exists(img_path_paddle):
        print(f"Đang xử lý: {img_path_paddle}...")
        process_image_to_data(img_path_tesseract, DATA_MAP_TESSERACT, STATE_TESSERACT, 'Tesseract')
        process_image_to_data(img_path_paddle, DATA_MAP_PADDLE, STATE_PADDLE, 'Paddle')

rows = []
all_ids = set(DATA_MAP_PADDLE.keys()) | set(DATA_MAP_TESSERACT.keys())

def sort_key(x):
    try:
        return int(str(x))
    except:
        return str(x)

for sid in sorted(all_ids, key=sort_key):
    # Lấy Tiếng Trung (src) từ PADDLE Map
    paddle_data = DATA_MAP_PADDLE.get(sid, {"src": [], "tgt": []})
    src_text = " ".join(paddle_data["src"]).strip()
    
    # Lấy Tiếng Việt (tgt) từ TESSERACT Map
    tesseract_data = DATA_MAP_TESSERACT.get(sid, {"src": [], "tgt": []})
    tgt_text = " ".join(tesseract_data["tgt"]).strip()
    # print(f"{sid}\n")
    # print(f"{src_text}\n")
    # print(f"{tgt_text}\n")
    if src_text or tgt_text:
        rows.append({
            "src_id": sid,
            "src_lang": src_text,  # src: Paddle
            "tgt_lang": tgt_text   # tgt: Tesseract
        })
df = pd.DataFrame(rows)

# Lọc ID 
ID_START = 626
ID_END = 750
df = df[df['src_id'].str.isdigit()].copy()
df['src_id_int'] = df['src_id'].astype(int)
df = df[(df['src_id_int'] >= ID_START) & (df['src_id_int'] <= ID_END)]
df = df.sort_values('src_id_int').drop(columns=['src_id_int']).reset_index(drop=True)


Đang xử lý: ../image\PDF1/PDF1-250.png...
Đang xử lý: ../image\PDF1/PDF1-251.png...
Đang xử lý: ../image\PDF1/PDF1-252.png...
Đang xử lý: ../image\PDF1/PDF1-253.png...
Đang xử lý: ../image\PDF1/PDF1-254.png...
Đang xử lý: ../image\PDF1/PDF1-255.png...
Đang xử lý: ../image\PDF1/PDF1-256.png...
Đang xử lý: ../image\PDF1/PDF1-257.png...
Đang xử lý: ../image\PDF1/PDF1-258.png...
Đang xử lý: ../image\PDF1/PDF1-259.png...
Đang xử lý: ../image\PDF1/PDF1-260.png...
Đang xử lý: ../image\PDF1/PDF1-261.png...
Đang xử lý: ../image\PDF1/PDF1-262.png...
Đang xử lý: ../image\PDF1/PDF1-263.png...
Đang xử lý: ../image\PDF1/PDF1-264.png...
Đang xử lý: ../image\PDF1/PDF1-265.png...
Đang xử lý: ../image\PDF1/PDF1-266.png...
Đang xử lý: ../image\PDF1/PDF1-267.png...
Đang xử lý: ../image\PDF1/PDF1-268.png...
Đang xử lý: ../image\PDF1/PDF1-269.png...
Đang xử lý: ../image\PDF1/PDF1-270.png...
Đang xử lý: ../image\PDF1/PDF1-271.png...
Đang xử lý: ../image\PDF1/PDF1-272.png...
Đang xử lý: ../image\PDF1/PDF1-273

4. Hậu xử lí OCR

In [15]:
# Định nghĩa bộ lọc Regex
def is_vietnamese(text):
    if not isinstance(text, str) or text.strip() == "":
        return False
    
    # \u0020-\u007E: Bao gồm a-z, A-Z, 0-9, dấu câu chuẩn (!, @, #, ., ?, ...)
    # Phần còn lại: Các nguyên âm có dấu và chữ Đ/đ của tiếng Việt
    vietnamese_chars = (
        "àáạảãâầấậẩẫăằắặẳẵ"
        "èéẹẻẽêềếệểễ"
        "ìíịỉĩ"
        "òóọỏõôồốộổỗơờớợởỡ"
        "ùúụủũưừứựửữ"
        "ỳýỵỷỹ"
        "đ"
        "ÀÁẠẢÃÂẦẤẬẨẪĂẰẮẶẲẴ"
        "ÈÉẸẺẼÊỀẾỆỂỄ"
        "ÌÍỊỈĨ"
        "ÒÓỌỎÕÔỒỐỘỔỖƠỜỚỢỞỠ"
        "ÙÚỤỦŨƯỪỨỰỬỮ"
        "ỲÝỴỶỸ"
        "Đ"
    )
    
    # Logic: Chuỗi hợp lệ chỉ được chứa các ký tự nằm trong 2 nhóm này
    # LƯU Ý: Nếu văn bản của bạn có dấu nháy Unicode (“ ”), bạn phải thêm chúng vào pattern này
    # hoặc xử lý chúng trước khi lọc, nếu không dòng đó sẽ bị False.
    pattern = f'^[\u0020-\u007E{vietnamese_chars}]+$'
    
    return bool(re.match(pattern, text))

# --- BỔ SUNG LOGIC XÓA TRƯỚC CHỮ 写 ---
def trim_before_xie(text):
    if not isinstance(text, str):
        return text
    idx = text.find('写')
    if idx != -1:
        return text[idx:]
    return text

# Áp dụng xóa ký tự trước chữ '写' cho cột src_lang
df['src_lang'] = df['src_lang'].apply(trim_before_xie)

# ---------------------------------------
# ĐOẠN CODE GỐC CỦA BẠN TIẾP TỤC TẠI ĐÂY
# ---------------------------------------

# Drop các dòng bị thiếu (NaN) hoặc rỗng ở 1 trong 2 cột
# Chuyển chuỗi chỉ có khoảng trắng thành NaN 
df = df.replace(r'^\s*$', float('nan'), regex=True)
df.dropna(subset=['src_lang', 'tgt_lang'], inplace=True)

# Logic: Giữ lại dòng khi (Check Trung == True) và (Check Việt == True)
# Các dòng False sẽ bị loại bỏ
df_clean = df[
    df['src_lang'].apply(is_chinese) & 
    df['tgt_lang'].apply(is_vietnamese)
].copy()
df_clean.reset_index(drop=True, inplace=True)

# Xuất CSV
if not df_clean.empty:
    min_id = df_clean['src_id'].astype(int).min()
    max_id = df_clean['src_id'].astype(int).max()
else:
    min_id = max_id = 0

out_csv = os.path.join(OUT_DIR, f"PDF1_{min_id}_{max_id}.csv")
df_clean.to_csv(out_csv, index=False, encoding="utf-8-sig")
print("Saved:", out_csv)

Saved: ../OCR\PDF1_626_750.csv
